In [1]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

In [23]:
urls=["https://search.shopping.naver.com/catalog/17071491333?query=%EC%83%A4%EC%98%A4%EB%AF%B8%20%ED%95%B8%EB%93%9C%ED%8F%B0&NaPm=ct%3Dksbprw48%7Cci%3Df595b39463a8b95059f07039549bdc607c3ae638%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Dfc7c1532019eb6ceea7cc12c6c0b0e54752c2e40"]

In [24]:
for url in urls:
    
    shoppingmall_review = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[1]/ul/li[4]/a"

    shppingmall_review2 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[2]/div/div[2]/ul/li[4]/a"
#                                                                v                                ^
    point5 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[2]/a"
    point4 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[3]/a"
    point3 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[4]/a"
    point2 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[5]/a"
    point1 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[6]/a"
    points_path = [point5, point4, point3, point2, point1]

    title_path = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[2]/div[1]/em"
    text_path = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[2]/div[1]/p"

    header = {'User-Agent': ''}
    d = webdriver.Chrome('chromedriver.exe') # webdriver = chrome
    d.implicitly_wait(3)
    d.get(url)
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, "html.parser")

    a = d.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div[1]/h2")
    a = a.text #상품 제목


    elem = d.find_element_by_tag_name("body")
    elem.send_keys(Keys.PAGE_DOWN)
    sleep(2)

    d.find_element_by_xpath(shoppingmall_review).click()
    sleep(2)

    elem.send_keys(Keys.PAGE_DOWN)
    sleep(2)

    full_list = [[],[]]


    for i in range(len(points_path)):
        point = []
    
        d.find_element_by_xpath(shppingmall_review2).click()
        sleep(2)

    
        d.find_element_by_xpath(points_path[i]).click()
        print(points_path[i])
        sleep(2)
    
        page = 1
        starpoint = 5-i
    
        while True:
            html = d.page_source
            dom = BeautifulSoup(html, "lxml")
            titles_raw = dom.find_all("em", {"class":"reviewItems_title__39Z8H"})
            texts_raw = dom.find_all("p", {"class":"reviewItems_text__XIsTc"})


            try:
                d.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a["+str(page)+"]").click()

                sleep(1)
                if page == 11:
                    page = 2


                html = d.page_source
                dom = BeautifulSoup(html, "lxml")
                titles_raw = dom.find_all("em", {"class":"reviewItems_title__39Z8H"})
                texts_raw = dom.find_all("p", {"class":"reviewItems_text__XIsTc"})
            
            
                for text_raw in texts_raw:
                    full_list[0].append(text_raw.text)
                    full_list[1].append(starpoint)
                    point.append(starpoint)

                if len(texts_raw) != 20:
                    break
        
                page = page + 1
            
                if len(point) > 400:
                    break
            
                sleep(4)
            except:
                for text_raw in texts_raw:
                    full_list[0].append(text_raw.text)
                    full_list[1].append(starpoint)
                    point.append(starpoint)
                break



    d.quit()
            
    df = pd.DataFrame(full_list)
    df = df.T
    df.columns = ["review", "star"]
    df.to_csv("C:/690009/TP2/final_project/member/NTJ/crawlerNSstar/data/"+a+".csv", encoding='utf-8-sig')


/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[2]/a
/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[3]/a
/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[4]/a
/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[5]/a
/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[6]/a


In [25]:
df

,review,star
0,물건을 받고 알뜰폰 요금제로 하니 한달에 요금 4천원정도 나오겠네요. 몇년전 중국폰...,5
1,포코폰 사전예약으로 구매했습니다.다른 회사의 플래그쉽 모델의 3분의 1가격으로 비슷...,5
2,저렴한 가격에 주문 후 다음날 바로 받아볼수 있었습니다.실수로 배송지를 잘못 기입...,5
3,중국 휴대폰은 처음 구매하는지라 조금 망설이긴 했는데..일단 가격이 가격이다 보니 ...,5
4,이제 국산 제품에 놀아나지 않겠다쭝국 폰이라고 반신반의 했는데 음청 이쁘고 사향 지...,5
...,...,...
728,데이터테더링으로만해서 게임하려고 산건데실내에서 gps를 못잡네요집에있는 아이폰6는 ...,2
729,"포코폰 자급제 예판하는곳이 몇군데 있는데여기가 별다른 사은품은 없지만,카드청구할인엘...",2
730,와이파이 연동 문제가 있네요. 집 와이파이가 첫날부터 계속 연결됐다가 끊기길 반복하...,2
731,상품은 만족합니다 배송 신경좀 써주세요,2
